# Government Funding Space Projects - FAPESP

While at AEB, I was tasked to study and analyze the open library of projects funded by FAPESP, an institution from the state of São Paulo that provide funds to scientific research to universities and companies, so that the agency could start drafting a proposal of partnership with them.

So, in this work, I perform an EDA of the data available at their open library, in order to provide insights and rationale for the best course of action AEB could take regarding FAPESP

## Initializing the code

first things first, the tools we'll be using througout the work need to be imported to the workspace. In this case we'll be using _Pandas_ for data manipulation and mostly _Matplotlib_ for graphic generation and visualization

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

And also open the dataset

In [4]:
dataset = pd.read_excel('dataset-space-projects.xlsx')

## Visualizing the Dataset

To start the analysis, let's take an overview look at the dataframe.

In [6]:
dataset.head()

,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Data de Início,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Instituições no Exterior),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados,Unnamed: 30
0,21/12066-2,Desenvolvimento de Bateria Aeronáutica Modular...,Development of High-performance Modular Aerona...,Arthur Roza Augusto,NaN,NaN,NaN,NaN,NaN,Arthur Roza Augusto,...,2022-03-01,2024-02-29,NaN,NaN,NaN,NaN,"Atualmente, aeronaves modernas e a segurança d...","Today, modern aircraft and flight safety are c...",21/15032-1,
1,20/13779-0,Máscara de furação aeronáutica reconfigurável ...,Numerical control drill jig,Glauber Lopes Mosqueira,NaN,NaN,NaN,Electroimpact do Brasil Indústria Aeroespacial...,Jacareí,Glauber Lopes Mosqueira,...,2021-12-01,2023-07-31,NaN,NaN,NaN,NaN,Máscaras de furação são o instrumento mais com...,Drill jigs are the most common tools used in t...,NaN,
2,20/05837-0,Desenvolvimento de um transceptor UHF/VHF para...,Transceiver UHF/VHF for TT&C subsystems of a c...,Marcelo Henrique Essado de Morais,NaN,NaN,NaN,Essado de Morais Ltda. - ME,Franca,Marcelo Henrique Essado de Morais,...,2021-06-01,2022-02-28,NaN,NaN,NaN,NaN,Desenvolvimento de um transceptor VHF e UHF fu...,This document describes the Project Plan to de...,21/05993-4,
3,21/05993-4,Desenvolvimento de um transceptor UHF/VHF para...,Transceiver UHF/VHF for TT&C subsystems of a c...,Marcelo Henrique Essado de Morais,NaN,NaN,Essado de Morais Ltda. - ME,NaN,NaN,Marcelo Henrique Essado de Morais,...,2021-06-01,2022-02-28,NaN,NaN,NaN,NaN,Desenvolvimento de um transceptor VHF e UHF fu...,NaN,NaN,
4,20/05902-6,Inspeção dimensional portátil de furos por mét...,Portable dimensional inspection for holes by T...,Fabio do Monte Sena,NaN,NaN,NaN,Lume Tecnologia Ltda,São José dos Campos,Fabio do Monte Sena,...,2021-04-01,2022-03-31,NaN,NaN,NaN,NaN,"O volume de componentes, os requisitos dimensi...","The number of components, the dimensional requ...",21/02729-4; 21/02837-1; 21/03143-3; 21/08902-0,


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 31 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   N. Processo                                       208 non-null    object        
 1   Título (Português)                                208 non-null    object        
 2   Título (Inglês)                                   148 non-null    object        
 3   Beneficiário                                      208 non-null    object        
 4   Instituição                                       0 non-null      float64       
 5   Cidade Instituição                                0 non-null      float64       
 6   Instituição Parceira                              55 non-null     object        
 7   Empresa                                           152 non-null    object        
 8   Município                     

It is worth to say that this dataset contains all the projects related to aeroespace engeneering 

## Searching for core space technology

One of the requirements of this project was that it must be focused in core space technology. The space sector is so general and relates to so many diferent fields that narrowing that scope was more than necessary. So, since we were looking for projects related to core space technology, the next thing I needed to do was to filter the dataframe based on this requirement.

The strategy I adopted was to search for keywords related to the tech I was looking for in some descriptive columns

In [31]:
keywords = ['lançador', 'satélite', 'foguete', 'inpe']
contains_keyword_resumo = [any(keyword in description.lower() for keyword in keywords) for description in dataset['Resumo (Português)'].fillna('')]
contains_keyword_assuntos = [any(keyword in description.lower() for keyword in keywords) for description in dataset['Assuntos'].fillna('')]

#contains_keyword = []

#dataset[contains_keyword]

In [33]:
a = zip(contains_keyword_resumo, contains_keyword_assuntos)

In [36]:
for i, n in enumerate(a):
    if i < 10:
        print(n)
        print(sum(n))
    else:
        break

(False, False)
0
(False, False)
0
(False, False)
0
(False, False)
0
(False, True)
1
(True, True)
2
(True, True)
2
(False, False)
0
(True, True)
2
(True, True)
2


TypeError: 'bool' object is not iterable